In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from datasets import load_dataset

import math
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os

import hydra
from omegaconf import DictConfig, OmegaConf
from collections import defaultdict

from utils import FaceDataset
from model import SimpleUNet


In [19]:
cfg = OmegaConf.load("ddpm-config.yaml")
trainer_cfg = cfg['trainer_cfg']

model = SimpleUNet(trainer_cfg)
state_dict = torch.load('test-model-weights.pth')
model.load_state_dict(state_dict)
model.eval()

print(model.named_parameters())

/var/folders/0n/_3v5_w510g31fqgxq75l3gnr0000gn/T/ipykernel_5164/1388431787.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('test-model-weights.pt

<generator object Module.named_parameters at 0x17fd68940>


In [21]:
def check_dead_neurons(model, threshold=1e-6):
    neurons = {}
    for name, param in model.named_parameters():
        if 'weight' in name:
            neurons[name] = ((param.abs() < threshold).sum().item(), param.numel())
    return neurons

neurons = check_dead_neurons(model)
print("Dead neurons in each layer:")
for layer, vals in neurons.items():
    print(f"{layer}: {vals[0]} / {vals[1]}")


Dead neurons in each layer:
time_mlp.1.weight: 9 / 1024
conv0.weight: 0 / 1728
downs.0.time_mlp.weight: 240 / 4096
downs.0.conv1.weight: 71 / 73728
downs.0.transform.weight: 314 / 262144
downs.0.conv2.weight: 217 / 147456
downs.0.bnorm1.weight: 0 / 128
downs.0.bnorm2.weight: 0 / 128
downs.1.time_mlp.weight: 610 / 8192
downs.1.conv1.weight: 802 / 294912
downs.1.transform.weight: 5841 / 1048576
downs.1.conv2.weight: 1667 / 589824
downs.1.bnorm1.weight: 0 / 256
downs.1.bnorm2.weight: 0 / 256
downs.2.time_mlp.weight: 1334 / 16384
downs.2.conv1.weight: 650825 / 1179648
downs.2.transform.weight: 3426349 / 4194304
downs.2.conv2.weight: 1638007 / 2359296
downs.2.bnorm1.weight: 0 / 512
downs.2.bnorm2.weight: 0 / 512
downs.3.time_mlp.weight: 2747 / 32768
downs.3.conv1.weight: 3288353 / 4718592
downs.3.transform.weight: 16167215 / 16777216
downs.3.conv2.weight: 8147636 / 9437184
downs.3.bnorm1.weight: 0 / 1024
downs.3.bnorm2.weight: 0 / 1024
ups.0.time_mlp.weight: 1364 / 16384
ups.0.conv1.weight: